# Loading libs & repos

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!python -m pip install pyyaml==5.1
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 5.1 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-0turpk94
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-0turpk94
  Resolved https://github.com/facebookresearch/detectron2.git to commit 8c4a333ceb8df05348759443d0206302485890e0
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.

In [ ]:
from detectron2.utils.logger import setup_logger

setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import DatasetEvaluator
from detectron2.data.datasets import register_coco_instances
from PIL import Image
import os
import torch
import cv2


# Metrics UTILS

GET BBOX DATA




In [ ]:
def get_image_id(json_file, image_name):
  for image_entry in json_file["images"]:
    if image_entry["file_name"] == image_name:
      return image_entry["id"]
  return -1

def convert_json_bbox(bbox_data):
  converted_bbox_data = []
  for bbox in bbox_data:
    converted_bbox_data.append(
        (bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3])
    )
  return converted_bbox_data

def load_bbox_data(json_file_path, image_name):
  coco_json_bbox_data = []
  with open(json_file_path, "r") as read_file:
    json_file = json.load(read_file)
  image_id = get_image_id(json_file, image_name)
  if image_id >= 0:
    for annotation in json_file["annotations"]:
      if annotation["image_id"] == image_id:
        coco_json_bbox_data.append(tuple(annotation["bbox"]))
  # returns data converted from coco json format
  return convert_json_bbox(coco_json_bbox_data)

def IOU(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 0
    boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))
    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou

def calculate_metrics(ground_truth, prediction_data):
    IOUs = []
    true_positive = 0 # correct detection
    false_positive = 0 # incorrect detection
    false_negative = 0 # missing values

    true_positive_treshold = 0.6
    false_positive_treshold = 0.6

    for true_bbox in ground_truth:
        max_iou = 0
        for pred_bbox in prediction_data:
            iou = IOU(true_bbox, pred_bbox)
            max_iou = max(iou, max_iou)
        if max_iou >= true_positive_treshold:
            true_positive += 1
            IOUs.append(max_iou)
        else:
            false_negative += 1
            IOUs.append(0)

    for pred_box in prediction_data:
        max_iou = 0
        for true_bbox in ground_truth:
            iou = IOU(pred_box, true_bbox)
            max_iou = max(iou, max_iou)
        if max_iou <= false_positive_treshold:
            false_positive += 1
    precision = float(true_positive) / (true_positive + false_positive) if (true_positive + false_positive) > 0. else 0.
    recall = float(true_positive) / (true_positive + false_negative) if (true_positive + false_negative) > 0. else 0.
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0. else 0
    mean_iou = sum(IOUs) / len(IOUs)

    return mean_iou, precision, recall, f1_score

# Model

In [ ]:
DRIVE_PATH = "/content/gdrive/MyDrive/magisterka"
MODELS_PATH = os.path.join(DRIVE_PATH, "models")


In [ ]:
!ls $MODELS_PATH

clear_faster.pth  rcnn_rotate.pth	Retina_only_rotate.pth
model_final.pth   rcnn_saturation.pth	Retina_saturation.pth
rcnn_augment.pth  Retina_augmented.pth
rcnn.pth	  RetinaNet50.pth


In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2 import model_zoo


MODEL_DETECTRON2_PATH = os.path.join(MODELS_PATH, "clear_faster.pth")
PREDICTION_THRESHOLD_DETECTRON = 0.7


class Detectron2Model:
    """Detectron2 model for object detection."""

    def __init__(self, model_path=MODEL_DETECTRON2_PATH, threshold = PREDICTION_THRESHOLD_DETECTRON, **kwargs):
        self.model_path = model_path
        self.threshold = threshold
        self.cfg = self.get_model_config(model_path=self.model_path, **kwargs)
        self.model = self.get_model(model_config=self.cfg)
        self.model.training = False

    def get_model_config(
        self,
        model_path,
        num_classes=2,
        **kwargs,
    ):
        self.cfg = get_cfg()
        self.cfg.MODEL.DEVICE = "cpu"
        self.cfg.merge_from_file(
            model_zoo.get_config_file(

              #  "COCO-Detection/retinanet_R_50_FPN_1x.yaml"
               "COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"
            )
        )
        # "COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"

        self.cfg.MODEL.WEIGHTS = model_path
        self.cfg.MODEL.ROI_HEADS.SCORE_TRESH_TEST = self.threshold
        self.cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes
       # self.cfg.MODEL.RETINANET.NUM_CLASSES=2
        return self.cfg

    def set_trainer(self, trainer):
        self.trainer = trainer

    def get_model(self, model_config):
        predictor = DefaultPredictor(model_config)
        return predictor

    def get_evaluator(self, dataset):
        evaluator = COCOEvaluator(dataset, cfg, False, output_dir="./output/")
        return evaluator

    def set_threshold(self, threshold):
        self.cfg.MODEL.ROI_HEADS.SCORE_TRESH_TEST = threshold

    def get_threshold(self):
        return self.cfg.MODEL.ROI_HEADS.SCORE_TRESH_TEST

    def predict(self, image, threshold = 0.85):
        prediction = self.model(image)
        prediction_bboxes = self.__get_bbox_data(prediction, threshold)
        return prediction_bboxes

    def __get_bbox_data(self, prediction, threshold):
        response = []
        boxes = prediction["instances"].pred_boxes
        scores = prediction["instances"].scores
        for idx, entry in enumerate(boxes):
          if scores[idx] >= threshold:
            entry = entry.detach().to("cpu").numpy()
            x0, y0, x1, y1 = entry
            response.append((x0, y0, x1, y1, scores[idx].item()))
        return response

ModuleNotFoundError: ignored

In [ ]:
model_detectron = Detectron2Model()

[09/25 15:44:11 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/gdrive/MyDrive/magisterka/models/clear_faster.pth ...


# MEAN AVERAGE PRECISION

In [ ]:
import torch


def intersection_over_union(boxes_preds, boxes_labels):
    box1_x1 = boxes_preds[0:1]
    box1_y1 = boxes_preds[1:2]
    box1_x2 = boxes_preds[2:3]
    box1_y2 = boxes_preds[3:4]
    box2_x1 = boxes_labels[0:1]
    box2_y1 = boxes_labels[1:2]
    box2_x2 = boxes_labels[2:3]
    box2_y2 = boxes_labels[3:4]

    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)

    # Need clamp(0) in case they do not intersect, then we want intersection to be 0
    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)
    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

    return intersection / (box1_area + box2_area - intersection + 1e-6)

In [ ]:
import torch
from collections import Counter

def mean_average_precision(
    pred_boxes, true_boxes, iou_threshold=0.5
):

    # Adapted from https://github.com/aladdinpersson/Machine-Learning-Collection/blob/master/ML/Pytorch/object_detection/metrics/mean_avg_precision.py
    # list storing all AP for respective classes
    average_precisions = []

    # used for numerical stability later on
    epsilon = 1e-6

    detections = list(pred_boxes)
    ground_truths = list(true_boxes)

    # sort by box probabilities which is index 2
    detections.sort(key=lambda x: x[4], reverse=True)
    TP = torch.zeros((len(detections)))
    FP = torch.zeros((len(detections)))
    total_true_bboxes = len(ground_truths)

    for detection_idx, detection in enumerate(detections):
        ground_truth_img = [
            bbox for bbox in ground_truths
        ]

        num_gts = len(ground_truth_img)
        best_iou = 0

        for idx, gt in enumerate(ground_truth_img):
            iou = intersection_over_union(
                torch.tensor(detection),
                torch.tensor(gt)
            )

            if iou > best_iou:
                best_iou = iou
                best_gt_idx = idx

        if best_iou > iou_threshold:
            TP[detection_idx] = 1
        # if IOU is lower then the detection is a false positive
        else:
            FP[detection_idx] = 1
    TP_cumsum = torch.cumsum(TP, dim=0)
    FP_cumsum = torch.cumsum(FP, dim=0)
    recalls = TP_cumsum / (total_true_bboxes + epsilon)
    precisions = TP_cumsum / (TP_cumsum + FP_cumsum + epsilon)
    precisions = torch.cat((torch.tensor([1]), precisions))
    recalls = torch.cat((torch.tensor([0]), recalls))
    # torch.trapz for numerical integration
    average_precisions.append(torch.trapz(precisions, recalls))
    return (sum(average_precisions) / len(average_precisions)).item()

# Prev sources

In [ ]:
# Model evaluation - mean (IOS,precisions,recalls,f1_scores)  over images
THRESHOLD = 0.7

def model_evaluate(model,# Model instance
                   threshold,# list of thresholds to be validated
                   dataset_path,# dataset absolute path
                   image_data,# list of images (annotations need to be read from .json by image name)
                   data_annotation,
                   net_type="detectron2"):# name of annotations json

    def average(entries):
      return sum(entries) / len(entries)

    #json_annotation_file = os.path.join(dataset_path, data_annotation)
    json_annotation_file = data_annotation
    #
    ious = []
    precisions = []
    recalls = []
    f1_scores = []
    #
    for image in image_data:
      image_absolute_path = os.path.join(dataset_path, image)
      if net_type == "detectron2":
        opencv_image = cv2.imread(image_absolute_path)
        prediction = model.predict(opencv_image, threshold = threshold)
      ref_bbox = load_bbox_data(json_annotation_file, image)
      iou_over_image, precision_over_image, recall_over_image, f1_over_image = calculate_metrics(ref_bbox, prediction)
      ious.append(iou_over_image)
      precisions.append(precision_over_image)
      recalls.append(recall_over_image)
      f1_scores.append(f1_over_image)

    return average(ious), average(precisions), average(recalls), average(f1_scores)

# TEST MEAN AVERAGE PRECISION

In [ ]:
def calculate_mAP_over_dataset(predictions, original_bboxs, iou_threshold = 0.5):
  # predictions -> list of predictions over dataset
  # original_bboxs -> list of original annotations for every image
  # threshold -> threshold for iou
  if len(predictions) != len(original_bboxs):
    raise Exception("Prediction and original annotations mismatch")
  mAPs = []
  for idx, pred in enumerate(predictions):
    ground_truth = original_bboxs[idx]
    mAP_for_image = mean_average_precision(pred, ground_truth, iou_threshold = iou_threshold)
    mAPs.append(mAP_for_image)
  #return mAPs
  return sum(mAPs) / len(mAPs)

# Metrics eval

In [ ]:
IMAGE_PATH = os.path.join(DRIVE_PATH, "test_images")
#IMAGE_RAW_PATH=os.path.join(DRIVE_PATH, "dataset")
ANNOTATIONS_PATH = os.path.join(IMAGE_PATH, "_annotations.coco.json")



In [ ]:
image_names = [image for image in os.listdir(IMAGE_PATH) if "json" not in image]
image_paths = [os.path.join(IMAGE_PATH, image) for image in os.listdir(IMAGE_PATH) if "json" not in image]
annotations_file = ANNOTATIONS_PATH


In [ ]:

# DETECTRON2
detectron2_preds = []
original_annotations = []

for image_name in image_names:
  bbox_data = load_bbox_data(annotations_file, image_name)
  original_annotations.append(bbox_data)

for image_path in image_paths:
  image = cv2.imread(image_path)
  prediction = model_detectron.predict(image, threshold=0.2)
  detectron2_preds.append(prediction)


detectron_mAPs = calculate_mAP_over_dataset(detectron2_preds, original_annotations, iou_threshold = 0.6)
metrics_detectron = model_evaluate(model_detectron, PREDICTION_THRESHOLD_DETECTRON, IMAGE_PATH, image_names, annotations_file, net_type = "detectron2")

#
print("mAP Detectron : {:.4f}".format(detectron_mAPs))
print(f"Mean IOUs over threshold : {PREDICTION_THRESHOLD_DETECTRON} ->", "{:.4f}".format(metrics_detectron[0]))
print(f"Mean prec over thresholds : {PREDICTION_THRESHOLD_DETECTRON}->", "{:.4f}".format(metrics_detectron[1]))
print(f"Mean recall over thresholds : {PREDICTION_THRESHOLD_DETECTRON} ->", "{:.4f}".format(metrics_detectron[2]))
print(f"Mean f1 over thresholds : {PREDICTION_THRESHOLD_DETECTRON} ->", "{:.4f}".format(metrics_detectron[3]))
#



mAP Detectron : 0.8095
Mean IOUs over threshold : 0.7 -> 0.5946
Mean prec over thresholds : 0.7-> 0.8276
Mean recall over thresholds : 0.7 -> 0.7564
Mean f1 over thresholds : 0.7 -> 0.7836
